In [37]:
from keras.applications.inception_v3 import InceptionV3 
from keras.preprocessing import image
from keras.models import Model, load_model 
from keras.callbacks import ModelCheckpoint 
from keras.layers import Dense, GlobalAveragePooling2D 
from keras.preprocessing.image import ImageDataGenerator 
from keras.optimizers import SGD 
import warnings
warnings.filterwarnings("ignore")
import os. path

MODEL_FILE = "flowers. hd5"

# Create a model if none exists. Freezes all training except in
# newly attached output layers. We can specify the number of nodes
# in the hidden penultimate layer, and the number of output
# categories.
def create_model(num_hidden, num_classes):
    base_model = InceptionV3 (include_top= False, weights ='imagenet')
    x = base_model.output
    x = GlobalAveragePooling2D() (x)
    X = Dense(num_hidden, activation='relu') (x)
    predictions = Dense(num_classes, activation='softmax') (x)
    

    
    for layer in base_model.layers:
        layer.trainable = False
    
    model = Model(inputs=base_model.input, outputs = predictions)
    return model


# Loads an existing model file, then sets only the last 
# # 3 layers (which we added) to trainable.
def load_existing (model_file):
    # Load the model 
    model = load_model(model_file)
    # Set only last 3 layers as trainable 
    numlayers = len(model.layers)
    for layer in model.layers[:numlayers-3]:
        layer.trainable = False
    for layer in model.layers[numlayers-3:]:
        layer.trainable = True
    return model


def train(model_file, train_path, validation_path, num_hidden=200, num_classes=5,steps=32, num_epochs=20):
    if os.path.exists(model_file):
        print("\n*** Existing model found at %s. Loading.***\n\n" % model_file)
        model = load_existing (model_file) 
    else:
        print("\n*** Creating new model ***\n\n")
        model = create_model(num_hidden, num_classes)

    model.compile(optimizer='rmsprop', loss='categorical_crossentropy'),
    # Create a checkpoint 
    checkpoint = ModelCheckpoint(model_file)
    train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip = True)

    test_datagen = ImageDataGenerator (rescale=1./255)

    train_generator = train_datagen.flow_from_directory(train_path, target_size=(249, 249), batch_size=32, class_mode = "categorical")

    validation_generator = test_datagen.flow_from_directory(validation_path, target_size=(249,249), batch_size=32, class_mode='categorical')

    model.fit( train_generator, steps_per_epoch = steps, epochs = num_epochs, callbacks = [checkpoint], validation_data = validation_generator, validation_steps = 50)
    # Train last two layers 
    for layer in model.layers[:249]:
        layer.trainable = False
    for layer in model.layers [249:]:
        layer.trainable = True
    model.compile(optimizer = SGD(lr=0.00001, momentum=0.9), loss='categorical_crossentropy')

def main():
    train(MODEL_FILE, train_path="flower_photos", validation_path="flower_photos", steps=120, num_epochs=10) 
if __name__ == "__main__":
    main()


*** Existing model found at flowers. hd5. Loading.***


Found 3670 images belonging to 5 classes.
Found 3670 images belonging to 5 classes.
Epoch 1/10
115/120 [===========================>..] - ETA: 16s - loss: 0.4316WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 1200 batches). You may need to use the repeat() function when building your dataset.


INFO:tensorflow:Assets written to: flowers. hd5/assets


INFO:tensorflow:Assets written to: flowers. hd5/assets


120/120 [==============================] - 555s 5s/step - loss: 0.4316 - val_loss: 0.3926


In [39]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [51]:
from keras.models import load_model 
import tensorflow as tf 
from tensorflow.python.keras.backend import set_session 
import numpy as np 
from PIL import Image 
from os import listdir 
from os. path import join
import warnings
warnings.filterwarnings("ignore")

MODEL_NAME='flowers. hd5'
# Our samples directory 
SAMPLE_PATH = 'samples'
dict={0:'daisy', 1:'dandelion'}
# Takes in a loaded model, an image in numpy matrix format, 
# And a label dictionary
session = tf.compat.v1.Session(graph = tf.compat.v1.Graph())
def classify(model, image):
    with session.graph.as_default():
        set_session(session) 
        result = model.predict(image) 
        themax = np.argmax(result)

    return (dict[themax], result[0][themax], themax)

# Load image 
def load_image(image_fname):
    img = Image.open(image_fname) 
    img = img.resize((249, 249)) 
    imgarray = np.array(img)/255.0 
    final = np.expand_dims(imgarray, axis=0) 
    return final


#Test main 
def main(): 
    with session.graph.as_default():
        set_session(session) 
        model = load_model(MODEL_NAME)
        sample_files = listdir(SAMPLE_PATH)
    for filename in sample_files:
        filename = join(SAMPLE_PATH, filename) 
        img = load_image(filename) 
        label, prob,_ = classify(model, img)
        print("We think with certainty %3.2f that image %s is %s."%(prob, filename, label))
if __name__=="__main__":
    main()

IsADirectoryError: [Errno 21] Is a directory: 'samples/daisy'